In [14]:
import ssl, socket
from urllib.parse import urlparse
import urllib 
from dateutil.parser import parse
import datetime
import socket
import whois, sys
from datetime import datetime
import pandas as pd
import numpy as np
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import tldextract
import re
from tld import get_tld
import requests
from urllib.request import Request, urlopen

In [15]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
# 쓰인 classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
# 활용 모듈 및 메소드
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression

In [16]:
#Feature01
#긴 URL
#0이 피싱, 1이 정상
def URL_Length(url):
    if len(url) <= 74:
        return 1
    else:
        return 0

In [17]:
def Shortening_Service(url):
    try :
        response = requests.get(url)
        for resp in response.history:
            if response.status_code == 301 or response.status_code == 302 :
                return 0
            else :
                return 1
    except :
        print("Exception")
        return 0

In [18]:
# 3. URL에 @가 유무
def having_At_Symbol(url):
    try:  
        if '@' in url:
            return 0
        else:
            return 1
    except:
        print("Exception")
    
    return 0

In [19]:
#Feature04
#// 리다이렉션
def double_slash_redirecting(url) :
    parse = urlparse(url)
    path = parse.path
    if '//' in path :
        return 0
    else :
        return 1

In [20]:
def Prefix_suffix(url):
    if '-' in url :
        return 0
    else :
        return 1

In [21]:
# 6. 서브도메인이 2개 이상인지
def having_Sub_Domain(url):
    try:
        url = remove_www(url)
        domain = get_tld(url, as_object=True)
        if domain.subdomain == "":
            return 1
        dot = domain.subdomain.count('.')
        # dot = 0일때 의심사이트로 분류하는게 더 좋긴 함.
        if dot >= 1:
            return 0
        else:
            return 1
        
    except:
        return 0

def remove_www(url):
    if 'www.' in url[:12]:
        url = url.replace("www.", "")
    return url

In [22]:
def Domain_registration_length(url):
    try :
        total_date = get_total_date(url)
        if total_date > 365:
            return 1
        else:
            return 0
    except whois.parser.PywhoisError:
        print("Exception")
        return 0
def get_total_date(url):
    try:
        domain = whois.whois(url)
        if type(domain.expiration_date) is list :
            expiration_date = domain.expiration_date[0]
        else :
            expiration_date = domain.expiration_date
        if type(domain.updated_date) is list :
            updated_date = domain.updated_date[0]
        else :
            updated_date = domain.updated_date
        
        total_date = (expiration_date - updated_date).days
    
        return total_date
    except :
        print("Exception")
        return 0

In [23]:
def favicon(url):
    try:
        resp = urlopen(url, timeout = 10)
        bs = BeautifulSoup(resp)
        parse_url = urlparse(url)
        tld = get_tld(url, as_object = True)
        tag_link = bs.findAll("link", rel=re.compile("^shortcut icon$", re.I))
        if fav_ico(url) is 1:
            return 1
        if not tag_link :
            tag_link = bs.findAll("link", rel=re.compile("^icon$", re.I))
        if not tag_link :
            return 0
        for link in tag_link:
            fav = link.get('href')
            parse_fav = urlparse(fav)

            if parse_fav.hostname == None :
                return 1
            elif tld.domain in fav :
                return 1
        return 0
    except:
        return 0
    
def fav_ico(url):
    try:
        http = httplib2.Http(timeout = 10)
        status, response = http.request(url+'/favicon.ico')
        if status.status == 200:
            return 1
        else:
            return 0
    except:
        return 0

In [24]:
#Feature13
#<a>태그에 링크
def getAnchorResult(url):
    try:
        tmp = tldextract.extract(url).domain
        positiveAnchor = 0
        negativeAnchor = 1
        soup = BeautifulSoup(urlopen(url, timeout = 10), "html.parser", from_encoding = 'iso-8859-1')
        for a_tag in soup.findAll("a"):
            if 'href' in a_tag.attrs:
                tldObj = tldextract.extract(a_tag.attrs['href'])
                if ( (tldObj.domain == "") | (tldObj.domain == " ") | (tldObj.domain == None)):
                    continue
                #print(tldObj.domain)
                if ( tldObj.domain == tmp):
                    positiveAnchor += 1
                else:
                    negativeAnchor += 1
        ratio = negativeAnchor /(positiveAnchor + negativeAnchor)
        if ratio > 0.67:
            return 0
        else:
            return 1
    except:
        return 0 

In [25]:
def getLinksInTags(url):
    try :
        parsed_uri = urlparse(url)
        domain = '{uri.netloc}'.format(uri = parsed_uri)
        http = httplib2.Http(timeout = 5)
        status, response = http.request(url)
        metaTags = BeautifulSoup(response, parse_only=SoupStrainer(['meta','script','link']))
        matchedDomains =0
        unMatchedDomains =0
        for tag in metaTags:
            content =""
            if tag.has_attr('content'):
                content += (tag['content'])
            if tag.has_attr('src'):
                content += (tag['src'])
            if tag.has_attr('link'):
                content += (tag['link'])
            matchObj = re.match(r'([^a-zA-Z\d]+|http[s]?://)?([a-z0-9|-]+)\.?([a-z0-9|-]+)\.([a-z0-9|-]+)',content,re.M|re.I)
            if matchObj:
                subdomain = matchObj.group(2)
                midDomain = matchObj.group(3)
                topDomain = matchObj.group(4)
                if domain.find(midDomain) != -1:  #we have a url that matches the domain of the site
                    matchedDomains +=1
                else:
                    unMatchedDomains +=1

        percentUnmatched = unMatchedDomains/(matchedDomains+unMatchedDomains)

        return percentUnmatched
    except httplib2.ServerNotFoundError:
            print("exc getLinksInTags Site is Down")
            retVal = 0
            pass
    except:
            print("exc getLinksInTags No tags were returned.  Setting to one")
            retVal = 1
            pass
    return retVal

In [26]:
# 15. form 태그의 action 속성 내에 공백이나 about:blank가 있는지
def form_action_Handler(url):
    try:
        http = httplib2.Http(timeout = 5)
        status, response = http.request(url)
        tags = BeautifulSoup(response, parse_only=SoupStrainer(['form'], timeout = 5))

        if tags.get('action') == "" or tags.get('action') == "about:blank":
            return 0
        else:
            return 1
    except:
        print("Exception")
        return 0

In [27]:
#Feature16
#Whois에 등록되지 않은 url
def abnormalUrl(url):
    try:
        domain = whois.whois(url)
    except :
        print("Exception")
        return 0
    return 1

In [28]:
def External_Load_script(url):
    try:
        http = httplib2.Http(timeout = 5)
        status, response = http.request(url)
        tags = BeautifulSoup(response, parse_only=SoupStrainer(['script']))
        good = 0
        bad = 0
        if tags.get('src'):
            bad += 1
        else:
            good += 1
        if bad / (bad+good) == 1:
            return 0
        else:
            return 1
    except:
        print("Exception")
        return 0

In [29]:
# 18. redirection 횟수
def count_Redirection(url):
    try:
        count = 0
        res = requests.head(url, allow_redirects=True, timeout = 10)
        #request
        for resp in res.history:
            print(resp)
            if resp.status_code == 301 or resp.status_code == 302:
                count += 1
        if count >= 3:
            return 0
        else:
            return 1
    except:
        print("Exception")
        return 0

In [30]:
#Feature19
#하이퍼링크에 마우스를 가져다 댈 시 상태표시줄 조작
def onMouseOver(url):
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, "html.parser")
        for a_tag in soup.findAll("a"):
            #print(a_tag.attrs)
            if 'onmouseover' in a_tag.attrs:
                #print(a_tag.attrs['onmouseover'])
                match = re.search('window.status',a_tag.attrs['onmouseover'])
                if match:
                    return 0
        return 1
    except:
        print("Exception")
        return 0   

In [31]:
# 21. Using_Pop_up_Window
def Prompt_in_Popup(url):
    try:
        res = requests.get(url)
        bs = BeautifulSoup(res.content)
        for tag in bs.findAll('script'):
            prompt = str(tag)
            matching = re.search(r'.*open\(|prompt\(.*', prompt)
            if matching:
                return 0
            else:
                return 1
        return 0
    except:
        print("Exception")
        return 0

In [32]:
#Feature22
#소스코드내 도메인 이름 존재
def DomainName_in_Source(url):
    try:
        res = requests.get(url)
        if tldextract.extract(url).domain in res.text :
            return 1
        return 0   
    except:
        print("Exception")
        return 0

In [33]:
#23
def Length_of_Source(url):
    try:
        res = requests.get(url)
        if len(res.text)> 50000:
            return 1
        else:
            return 0
    except :
        return 0

In [34]:
# 24. duplicated_Tag
def duplicated_Tag(url):
    res = requests.get(url)
    head = res.text.count("</head>")
    body = res.text.count("</body>")
    
    if head >= 2 or body >= 2:
        return 0
    else:
        return 1   

In [35]:
#Feature25
#남은 도메인 수명
 
def Remain_Expiration(url) :
    try :
        total_date = get_remain_date(url)
        if total_date >= 180:
            return 1
        else:
            return 0
    except whois.parser.PywhoisError :
        print("Exception")
        return 0
    
def get_remain_date(url) :
    try:
        domain = whois.whois(url)
        if type(domain.expiration_date) is list :
            expiration_date = domain.expiration_date[0]
        elif domain.expiration_date == None :
            return 0
        else :
            expiration_date = domain.expiration_date
        total_date = (expiration_date - datetime.today()).days
        return total_date
    except :
        return 0

In [36]:
def Disabling_Right_Click(url):
    try:
        res = requests.get(url)
        if "event.button==2" in res.text:
            return 0
        else :
            return 1
    except :
        return 0

In [37]:
# 27. Web_Traffic 순위(알렉사 데이터베이스 참고)
def Alexa_Ranking(url):
    try:
        http = httplib2.Http()
        status, response = http.request("http://data.alexa.com/data?cli=10&dat=s&url="+url)
        rank = BeautifulSoup(response, 'xml').find("REACH")["RANK"]

        if not rank:
            return 0
        else:
            return 1
    except:
        return 0